In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
!pip install statsmodels
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("movies.csv")
data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 3.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,407045464.0,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,475766228.0,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
4,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,269000000.0,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722644,237033,The Fistic Mystic,Animation,en,Popeye and Olive enter the city of Badgag and ...,0.600,Famous Studios,1946-11-28,0.0,0.0,6.0,Released,NaN,7.000,2.0,Harry Welch-Mae Questel-Jackson Beck,NaN,/aaCKbhr13sYBXPa5DLzz8lByE0.jpg,/sXAGQoSwUEkLUATVruswxmVIh1o.jpg,NaN
722645,499883,The Killers,NaN,ru,Mohamed Aboulouakar’s diploma film from VGIK i...,0.600,NaN,1971-01-01,0.0,0.0,20.0,Released,NaN,0.000,0.0,NaN,NaN,NaN,NaN,NaN
722646,418960,Island of Love,Drama,id,Widowed lighthouse keeper Maria (Suzanna) and ...,0.600,NaN,1978-01-01,0.0,0.0,107.0,Released,NaN,0.000,0.0,Suzzanna-Nur Afni Octavia-Parto Tegal-Moh. Moc...,NaN,NaN,NaN,NaN
722647,948159,Shania Twain - by Stetson,Music,en,NaN,0.600,NaN,2005-02-18,0.0,0.0,0.0,Released,NaN,0.000,0.0,Shania Twain,NaN,/3v5Jpum4Jphvs7bY1fumm2BPPsR.jpg,NaN,NaN


In [ ]:
data.shape

(722649, 20)

In [ ]:
data.columns

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations'],
      dtype='object')

Goal: Predicting Revenue (total all time)

Model Assignments (tdb):

Napoleon: Linear Regression

In [ ]:
# Assuming 'X' is your feature matrix and 'y' is your target variable
# X and y should be NumPy arrays or pandas DataFrames/Series
X = data.loc[:, data.columns != 'revenue']
y = data.loc[:, data.columns == 'revenue']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train = pd.concat([X_train, y_train], axis=1)
# len(train) # 578119
len(train.loc[train['vote_average'] == 0]) # 564444

# min(train['popularity'])


341120

VIF (Variance Inflation Factors) to remove non relevant columns

In [ ]:
# calculate Variance Inflation Factor for each explanatory variable
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# The dataframe passed to VIF must include the intercept term. We add it the same way we did before.
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

# cols = ['Unemployment', 'AccordQueries', 'CPIEnergy', 'CPIAll', 'MilesTraveled']
# VIF(honda_train, cols)
# THROW AWAY VIF Values that are above 5, but remove one variable one at a time and rechecking VIF values.

In [ ]:
data.columns

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations'],
      dtype='object')

In [ ]:
# ols stands for Ordinary Least Squares
model = smf.ols(
    formula='revenue ~ id + '\
            'title + genres + original_language + overview + popularity + '\
            'production_companies + release_date'\
            'budget + revenue + runtime + status + tagline + vote_average + vote_count +'\
            'credits + keywords + posteR_path + backdrop_path + recommendations',
    data=train).fit()

print(model.summary())

cols = ['Unemployment', 'AccordQueries', 'CPIEnergy', 'CPIAll', 'MilesTraveled']
VIF(honda_train, cols)